ETL 'movies_dataset'

In [71]:
# Paso 1: Agregar la ruta de funciones a sys.path
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'code')))


In [72]:
import sys
import os
print(os.path.abspath(os.path.join('..', 'code')) in sys.path)


True


Conversion movies_dataset.csv a formato parquet

In [73]:
from CSVtoPARQUET import convertir_csv_a_parquet

df = convertir_csv_a_parquet(
    csv_path='..//data/Raw/Movies-20241026T022341Z-001/Movies/movies_dataset.csv', 
    parquet_path='../data/processed_data/movies_dataset.parquet',
    columns_to_drop=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'],
    fillna_values={'revenue': 0, 'budget': 0},
    dtype_conversion={'popularity': float, 'budget': int, 'id': int}
)


/home/carla/Documentos/DataScienceProyecto1/DataScienceProyecto1/code/CSVtoPARQUET.py:12: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [74]:
import pandas as pd

df = pd.read_parquet(r'../data/processed_data/movies_dataset.parquet')
df.head(5)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,7.7,5415.0
1,None,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,None,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862.0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0


Eliminar los valores nulos del campo realese date:

In [75]:
df = df.dropna(subset=['release_date'])

Conversion de fechas a formato AAAA-mm-dd y creacion de la columna release_year para extraer el año de la fecha de estreno

In [76]:
# Identificar filas con valores incorrectos
incorrect_format = df[~df['release_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]
# Muestra las filas con formato incorrecto.
incorrect_format


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19730,0.065736,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",NaN,104.0,Released,NaN,False,6.0,1,0.0,NaN,None,None,None,None,NaN,NaN
29503,1.931659,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",NaN,68.0,Released,NaN,False,7.0,12,0.0,NaN,None,None,None,None,NaN,NaN
35587,2.185485,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",NaN,82.0,Released,NaN,False,4.3,22,0.0,NaN,None,None,None,None,NaN,NaN


In [77]:
# borrar filas con valores incorrectos
df.drop(incorrect_format.index, inplace=True)

In [78]:
# Conversión de 'release_date' y creación de 'release_year'
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

# Muestra aleatoria de 6 filas
df[['release_date', 'release_year']].sample(6)


,release_date,release_year
29992,2002-12-24,2002
14218,2009-01-23,2009
2320,1998-03-05,1998
26107,2012-04-27,2012
7086,1969-12-11,1969
10544,2004-08-10,2004


Creacion de la columna "return"con el retorno de inversion y que tenga los campos revenue y budget, dividiendo revenue/budget (cuando no hay datos disponibles para calcularlo se tomara el valor 0)

In [79]:
# Creación de la columna 'return' usando operaciones vectorizadas
df['return'] = (df['revenue'] / df['budget']).where(df['budget'] > 0, 0)

# Muestra aleatoria de 20 filas para verificar
df[['revenue', 'budget', 'return']].sample(20)


,revenue,budget,return
16112,0.0,0.0,0.000000
11426,0.0,0.0,0.000000
8995,0.0,0.0,0.000000
23469,0.0,0.0,0.000000
14240,69564.0,11500000.0,0.006049
39046,0.0,0.0,0.000000
35146,0.0,0.0,0.000000
32232,0.0,0.0,0.000000
25756,0.0,0.0,0.000000
25121,0.0,0.0,0.000000


Filtrar películas publicadas y eliminar la columna status

In [80]:
# Filtrar películas publicadas y eliminar la columna 'status'
df = df[df['status'] == 'Released'].drop('status', axis=1).reset_index(drop=True)


In [81]:
# Verificar la forma del DataFrame
print("Forma del DataFrame:", df.shape)

# Verificar que la columna 'status' ha sido eliminada
try:
    print("Valores únicos en 'status':", df['status'].unique())
except KeyError:
    print("La columna 'status' ha sido eliminada correctamente.")

# Contar las películas filtradas
print("Número de películas publicadas:", len(df))

# Inspeccionar las primeras filas
print("Primeras filas del DataFrame:")
print(df.head())

# Verificar el índice
print("Índice del DataFrame:", df.index)

# Verificar los tipos de datos
print("Tipos de datos de cada columna:")
print(df.dtypes)


Forma del DataFrame: (44936, 19)
La columna 'status' ha sido eliminada correctamente.
Número de películas publicadas: 44936
Primeras filas del DataFrame:
                               belongs_to_collection      budget  \
0  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000.0   
1                                               None  65000000.0   
2  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0.0   
3                                               None  16000000.0   
4  {'id': 96871, 'name': 'Father of the Bride Col...         0.0   

                                              genres       id  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...    862.0   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   8844.0   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  15602.0   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  31357.0   
4                     [{'id': 35, 'name': 'Comedy'}]  11862.0   

  original_language                           

Desanidar campos anidados: belongs_to_collection, genres, production_companies, production_countries, spoken_languages

Desanidar belongs_to_collection:

In [82]:
# Verificar la distribución de los tipos de datos en la columna
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40470
<class 'str'>          4466
Name: count, dtype: int64

In [83]:
# convertir la columa a una lista de diccionarios
from STRtoLISToDICT import convert_list_dict

df['belongs_to_collection'] = df['belongs_to_collection'].apply(convert_list_dict)

# Mostrar el DataFrame con los tipos corregidos
df[['belongs_to_collection']].head(6)

,belongs_to_collection
0,"{'id': 10194, 'name': 'Toy Story Collection', ..."
1,None
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
3,None
4,"{'id': 96871, 'name': 'Father of the Bride Col..."
5,None


In [84]:
# verificar cuántos valores en la columna belongs_to_collection son del tipo dict, list, o NoneType
df['belongs_to_collection'].apply(type).value_counts() 

belongs_to_collection
<class 'NoneType'>    40470
<class 'dict'>         4466
Name: count, dtype: int64

In [85]:
# Código para desanidar los diccionarios en la columna belongs_to_collection
belong_desanidado = pd.json_normalize(df['belongs_to_collection'])  # Convierte los diccionarios de cada fila en una tabla con columnas separadas.

# Renombrar las columnas para que tengan un prefijo 'btc_'
belong_desanidado.rename(columns={'id': 'btc_id',
                                  'name':'btc_name',
                                  'poster_path':'btc_poster_path',
                                  'backdrop_path': 'btc_backdrop_path'}, inplace=True)
belong_desanidado.head(6)

,btc_id,btc_name,btc_poster_path,btc_backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,NaN,NaN,NaN,NaN
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,NaN,NaN,NaN,NaN
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
5,NaN,NaN,NaN,NaN


In [86]:
# CombinaR EL DataFrame original df con el DataFrame belong_desanidado, añadiendo las columnas desanidadas a df.
df = pd.concat([df, belong_desanidado], axis=1)

In [87]:
# Verificar que la concatenación se realizó correctamente
df.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,tagline,title,vote_average,vote_count,release_year,return,btc_id,btc_name,btc_poster_path,btc_backdrop_path
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,None,Toy Story,7.7,5415.0,1995,12.451801,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,None,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035,NaN,NaN,NaN,NaN
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,None,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760,NaN,NaN,NaN,NaN
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862.0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [ ]:
#TODAVIA NO EJECUTE

# Eliminar columna belongs_to_collection
#df = df.drop('belongs_to_collection', axis=1)

Desanidar genres:

In [88]:
# Verificar la distribución de los tipos de datos en la columna.
df['genres'].apply(type).value_counts() 

genres
<class 'str'>    44936
Name: count, dtype: int64

In [89]:
# convertir la columa a una lista de diccionarios
from STRtoLISToDICT import convert_list_dict

df['genres'] = df['genres'].apply(convert_list_dict)

# Mostrar el DataFrame con los tipos corregidos
df[['genres']].head(6)

,genres
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,"[{'id': 35, 'name': 'Comedy'}]"
5,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."


In [90]:
# verificar cuántos valores en la columna genres son del tipo dict, list, o NoneType
df['genres'].apply(type).value_counts() 

genres
<class 'list'>    44936
Name: count, dtype: int64

In [91]:
# Eliminar NoneTypes (son valores nulos)
nulos = df[~df['genres'].apply(lambda x: isinstance(x, list))]
nulos

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,tagline,title,vote_average,vote_count,release_year,return,btc_id,btc_name,btc_poster_path,btc_backdrop_path


In [92]:
# Eliminar las filas en el DataFrame df que contienen valores nulos (NaN) en la columna genres
df = df.dropna(subset=['genres'])
# Verificacion de tipos y conteo de elementos
df['genres'].apply(type).value_counts()

genres
<class 'list'>    44936
Name: count, dtype: int64

In [93]:
print(df.columns)


Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count',
       'release_year', 'return', 'btc_id', 'btc_name', 'btc_poster_path',
       'btc_backdrop_path'],
      dtype='object')


In [94]:
from desanidarColumnas import desanidar_columna
# Aplicamos la funcion y ahora tenemos la tabla desanidada y la tabla original con los id pertenecientes de la tabla desanidada.
#Le asignamos el nombre que en realidad es un Dataframe a una variable.
genres_desanidado = desanidar_columna(df, 'genres', 'genres_desanidado')

genres_desanidado.head(4)

,id,name,genres_id
0,16.0,Animation,0
1,35.0,Comedy,0
2,10751.0,Family,0
3,12.0,Adventure,1


Desanidar production_companies

In [95]:
# Verificar la distribución de los tipos de datos en la columna.
df['production_companies'].apply(type).value_counts() 

production_companies
<class 'str'>    44936
Name: count, dtype: int64

In [96]:
# Convertir los valores de la columna en listas de diccionarios y realizar conteo de los tipos de datos en la columna
df['production_companies'] = df['production_companies'].apply(convert_list_dict)
df['production_companies'].apply(type).value_counts() 

production_companies
<class 'list'>    44936
Name: count, dtype: int64

In [97]:
# Desanidar columna aplicando la funcion para desanidar columnas
pc_desanidado = desanidar_columna(df, 'production_companies', 'pc_desanidado', 1)  
# El argumento 1 se utiliza para asegurarse de que los identificadores generados para esta tabla no entren en conflicto con genres_id.
pc_desanidado.head(6)

,name,id,production_companies_id
0,Pixar Animation Studios,3.0,1
1,TriStar Pictures,559.0,2
2,Teitler Film,2550.0,2
3,Interscope Communications,10201.0,2
4,Warner Bros.,6194.0,3
5,Lancaster Gate,19464.0,3


Desanidar production_countries

In [98]:
# Aplicar la Función convert_list_dict para convertir las listas de diccionarios  
df['production_countries'] = df['production_countries'].apply(convert_list_dict)
# Verificar tipos de datos
df['production_countries'].apply(type).value_counts()

production_countries
<class 'list'>    44936
Name: count, dtype: int64

In [99]:
# Desanidar la Columna production_countries
pctry_desanidado = desanidar_columna(df, 'production_countries', 'pctry_desanidado', 2) 
# 'production_countries': Columna anidada
#'pctry_desanidado': nombre de la tabla que se almacenara en la variable pctry_desanidado
# 2: Número identificador para que los id no se solapen con otros desanidamientos
pctry_desanidado.head(6)

,iso_3166_1,name,production_countries_id
0,US,United States of America,2
1,US,United States of America,3
2,US,United States of America,4
3,US,United States of America,5
4,US,United States of America,6
5,US,United States of America,7


Desanidar spoken_languages

In [100]:
# Aplicar la Función convert_list_dict
df['spoken_languages'] = df['spoken_languages'].apply(convert_list_dict)
# Verificar los Tipos de Datos
df['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'list'>    44936
Name: count, dtype: int64

In [101]:
# Desanidar la Columna utilizando la función desanidar_columna para crear una nueva tabla llamada slan_desanidado
slan_desanidado = desanidar_columna(df, 'spoken_languages', 'slan_desanidado', 3)  # 3: identificador
slan_desanidado.head(6)

,iso_639_1,name,spoken_languages_id
0,en,English,3
1,en,English,4
2,fr,Français,4
3,en,English,5
4,en,English,6
5,en,English,7


Optimizacion del tamaño del dataset y simplificar su estructura

In [102]:
# Eliminar Columnas que no son necesarias
df = df.drop('btc_poster_path', axis=1)
df = df.drop('btc_backdrop_path', axis=1)
df = df.drop('belongs_to_collection', axis=1)



In [103]:
# Eliminar una Columna de la frase celebre de una pelicula 
df = df.drop('tagline', axis=1)

In [104]:
#  eliminar las columnas anidadas
df = df.drop('genres', axis=1)
df = df.drop('production_companies', axis=1)
df = df.drop('production_countries', axis=1)
df = df.drop('spoken_languages', axis=1)

In [105]:
# Verificar el dataframe optimizado
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,btc_id,btc_name,genres_id,production_companies_id,production_countries_id,spoken_languages_id
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,1995,12.451801,10194.0,Toy Story Collection,0,1,2,3
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9,2413.0,1995,4.043035,NaN,NaN,1,2,3,4
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Grumpier Old Men,6.5,92.0,1995,0.000000,119050.0,Grumpy Old Men Collection,2,3,4,5
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1,34.0,1995,5.090760,NaN,NaN,3,4,5,6
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Father of the Bride Part II,5.7,173.0,1995,0.000000,96871.0,Father of the Bride Collection,4,5,6,7


Realizar modificaciones para el desarrollo de la API

In [106]:
# Preparar el DataFrame para la API
# Minúsculas en títulos
df['title'] = df['title'].str.lower()  # convierte todos los títulos de películas a minúsculas, útil para evitar problemas de comparación

In [ ]:
# Conversión de tipos de datos:
df['vote_count'] = df['vote_count'].astype(int)
df['id'] = df['id'].astype(int)

In [108]:
df['release_year'] = df['release_year'].astype(int)

In [109]:
# Redondeo de columnas
df['return'] = df['return'].round(2)
df['popularity'] = df['popularity'].round(2)

In [110]:
# Reinicio de índice
df = df.reset_index(drop=True)  # restablece el índice del DataFrame, útil para evitar problemas con índices desalineados

In [111]:
# Verificar el dataframe optimizado
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,btc_id,btc_name,genres_id,production_companies_id,production_countries_id,spoken_languages_id
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.95,1995-10-30,373554033.0,81.0,toy story,7.7,5415,1995,12.45,10194.0,Toy Story Collection,0,1,2,3
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.02,1995-12-15,262797249.0,104.0,jumanji,6.9,2413,1995,4.04,NaN,NaN,1,2,3,4
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.71,1995-12-22,0.0,101.0,grumpier old men,6.5,92,1995,0.00,119050.0,Grumpy Old Men Collection,2,3,4,5
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.86,1995-12-22,81452156.0,127.0,waiting to exhale,6.1,34,1995,5.09,NaN,NaN,3,4,5,6
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.39,1995-02-10,76578911.0,106.0,father of the bride part ii,5.7,173,1995,0.00,96871.0,Father of the Bride Collection,4,5,6,7


In [112]:
df.info()
'''Utilizar La función df.info() en pandas proporciona un resumen conciso del DataFrame, 
nos da información sobre: el índice(indica cómo está indexado el DataFrame), el número de entradas(número total de filas en el DataFrame), 
el tipo de datos de cada columna y la cantidad de valores no nulos en cada columna.'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44936 entries, 0 to 44935
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   budget                   44936 non-null  float64       
 1   id                       44936 non-null  int64         
 2   original_language        44926 non-null  object        
 3   overview                 44029 non-null  object        
 4   popularity               44936 non-null  float64       
 5   release_date             44936 non-null  datetime64[ns]
 6   revenue                  44936 non-null  float64       
 7   runtime                  44696 non-null  float64       
 8   title                    44936 non-null  object        
 9   vote_average             44936 non-null  float64       
 10  vote_count               44936 non-null  int64         
 11  release_year             44936 non-null  int64         
 12  return                   44936 n

'Utilizar La función df.info() en pandas proporciona un resumen conciso del DataFrame, \nnos da información sobre: el índice(indica cómo está indexado el DataFrame), el número de entradas(número total de filas en el DataFrame), \nel tipo de datos de cada columna y la cantidad de valores no nulos en cada columna.'

Load

In [114]:
# Asegúrate de que el directorio de destino exista
output_dir = '../data/processed_data/movies'
os.makedirs(output_dir, exist_ok=True)

Exportacion del DataFrame principal

In [115]:
# Definición de la ruta del archivo
parquet_path = os.path.join(output_dir, 'movies_dataset_etl.parquet')
# Exportación del DataFrame
df.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a '../data/processed_data/movies'")

Datos exportados correctamente a '../data/processed_data/movies'


Exportacion de las tablas extras 
(Se exportaran las tablas: 'genres_desanidado.parquet' y 'pc_desanidado.parquet'. 
Las tablas 'pctry_desanidado' y 'slan_desanidado' no se consideran necesarias')

In [116]:
# Definición de la ruta del archivo
parquet_path = os.path.join(output_dir, 'genres_desanidado.parquet')
# Exportación del DataFrame genres_desanidado
genres_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a '../data/processed_data/movies'")

Datos exportados correctamente a '../data/processed_data/movies'


In [ ]:
# Definición de la ruta
parquet_path = os.path.join(output_dir, 'pc_desanidado.parquet')
# Exportación del DataFrame pc_desanidado
pc_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a '../data/processed_data/movies'")

Datos exportados correctamente a '../data/processed_data/movies'
